## 1. Монета (10 баллов)

Крупье выбирает монету из двух в начале с равной вероятностью (0.5). 

Вероятности выпадения орла и решки на правильной монете - одинаковые (0.5).

Вероятность выпадения орла на неправильной монете - 0.9, решки - 0.1.

Крупье начинает подкидывать монету и выпадают только орлы.

После какого выпадения можно с уверенностью 95% говорить, что крупье выбрал неправильную монету.

Вероятность подкинуть $k$ орлов подряд в обычной монете:
$$P(O_k) = 0.5^k$$

Нужно найти такое минимальное $k$, что:
$$0.5^k \leq 0.05$$
$$k \geq \log_{0.5} 0.05$$
$$k = \lceil 4.32 \rceil = 5$$


## 1. PAM (10 баллов)

Выведите формулу для коэффициентов при возведении в n-ую степень следующей матрицы:
$$PAM = \begin{bmatrix}
    1-3a & a & a & a\\
    a & 1-3a & a & a\\
    a & a & 1-3a & a\\
    a & a & a & 1-3a\\
\end{bmatrix}$$
